In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import versor as vsr
from versor.drawing import *
from motor_estimation import MotorEstimationSolver
from game import (MotorEstimationSolver, VDMotorEstimationSolver)

# Rotor Estimation

In [3]:
rotor = vsr.Biv(pi/6, 0.0, 0.0).exp()
print(rotor)

Rot: [ 0.87 -0.5 0 0 ]


In [4]:
vectors_a = [vsr.Vec(1,1,0), vsr.Vec(1,0,1), vsr.Vec(0,1,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [5]:
initial_rotor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_rotor)
for a, b in zip(vectors_a, vectors_b):
    mes.add_point_correspondences_residual_block(a.null(),b.null())
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_rotor, summary, rotors) = mes.solve()
# print(summary['full_report'])
print(estimated_rotor)

Mot: [ 0.87 -0.5 -2.5e-10 -3.8e-10 -5.1e-10 -3.9e-10 3.9e-10 -2.3e-10 ]


In [6]:
scene = Scene(children=
              [vector_mesh(a,arrow=True,color='hotpink') for a in vectors_a] +\
              [vector_mesh(b,arrow=True,color='lightskyblue') for b in vectors_b] +\
              [frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')

display(renderer)

# Circles

In [7]:
motor = vsr.Vec(0.5,1,1.5).trs() * vsr.Biv(pi/6, 0.0, 0.0).exp()
p1s = [vsr.Vec(1,0,0).null(), vsr.Vec(0,1,0).null(), vsr.Vec(0,0,1).null()] 
p2s = [p1i.spin(motor) for p1i in p1s] 
p3s = [(p2i - p1i).vec().null() for p1i, p2i in zip(p1s,p2s)]
c1 = vsr.Cir(*p1s)
c2 = vsr.Cir(*p2s)
c3 = vsr.Cir(*p3s)
pp1 = c1.dual().split() 
(pp1[1] - pp1[0]).vec().norm() == 2.0 * c1.radius()

AttributeError: 'Dlp' object has no attribute 'vec'

In [8]:
c1.radius()

NameError: name 'c1' is not defined

In [9]:
print(p3s)

NameError: name 'p3s' is not defined

In [10]:
class Colors:
    RED = 'red'
    GREEN = 'green'
    BLUE = 'blue'
    HOTPINK = 'hotpink'
    LIGHTSKYBLUE = 'lightskyblue'
    AQUAMARINE = 'aquamarine'

In [11]:
Colors.LIGHTSKYBLUE

'lightskyblue'

In [12]:
scene = Scene(children=
              [point_mesh(p1i,color=Colors.HOTPINK) for p1i in p1s] +\
#               [point_mesh(p2i,color='lightskyblue') for p2i in p2s] +\
              
              [circle_mesh(c1, color='hotpink')] +\
              [point_mesh(p, color='red') for p in  pp1] +\
#               [circle_mesh(c2, color='lightskyblue')] +\
#               [circle_mesh(c3, color='turquoise')] +\
#               [circle_mesh(c2 - c1, color='red')] +\
              [line_mesh(vsr.Dll(*pp1), color=Colors.GREEN)] +\

              [frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

NameError: name 'c1' is not defined

In [13]:
pp = c1.dual()
pp.__print_debug_info_console()

NameError: name 'c1' is not defined

# Tangents

In [14]:
t1 = vsr.Tnv(vsr.Vec(1,2,3).null(), vsr.Vec(-1,1,1).unit())
t2 = vsr.Tnv(vsr.Vec(1,3,3).null(), vsr.Vec(-1,2,1).unit())
t3 = t2.comm(t1).dual()
print(t1.radius())
print(t2.radius())
print(t3.radius())

4.21468485109e-08
5.96046447754e-08
0.5


In [15]:
def tangent_vector_mesh(tnv, color='gray'):
    tangent = point_mesh(tnv.pnt(), color=color)
    tangent.children = [vector_mesh(tnv.dir().vec(), arrow=True, color=color)]
    return tangent
    

In [16]:
scene = Scene(children=
              [tangent_vector_mesh(t1, color=Colors.LIGHTSKYBLUE),
               tangent_vector_mesh(t2, color=Colors.LIGHTSKYBLUE),
               circle_mesh(t3, color=Colors.HOTPINK),
               frame_mesh(),
               AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [17]:
p = vsr.Par(vsr.Vec(1,2,3).null(), vsr.Vec(-1,1,1).null())
p

Tnv: [ 3 4 -1 2 1 2 8.5 -4 -2.5 -5.5 ]

In [18]:
a = vsr.Vec(1,0,0).null()
b = vsr.Vec(-1,0,0).null()
l = vsr.Dll(a,b)
l.dir()

Drv: [ -2 0 0 ]

In [19]:
vsr.Par(a,b).pnt()

Pnt: [ 0 0 0 1 0 ]

In [20]:
scene = Scene(children=
              [point_mesh(a), point_mesh(b)] +\
              [circle_mesh(t.dual(), color='hotpink')] +\

              [line_mesh(vsr.Dll(a,b), color=Colors.GREEN)] +\
              [frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

NameError: name 't' is not defined